In [1]:
# Code to decode Valhalla Geometries

import sys

#six degrees of precision in valhalla
inv = 1.0 / 1e6;

#decode an encoded string
def decode_geometry(encoded):
  decoded = []
  previous = [0,0]
  i = 0
  #for each byte
  while i < len(encoded):
    #for each coord (lat, lon)
    ll = [0,0]
    for j in [0, 1]:
      shift = 0
      byte = 0x20
      #keep decoding bytes until you have this coord
      while byte >= 0x20:
        byte = ord(encoded[i]) - 63
        i += 1
        ll[j] |= (byte & 0x1f) << shift
        shift += 5
      #get the final value adding the previous offset and remember it for the next
      ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
      previous[j] = ll[j]
    #scale by the precision and chop off long coords also flip the positions so
    #its the far more standard lon,lat instead of lat,lon
    decoded.append([float('%.6f' % (ll[0] * inv)), float('%.6f' % (ll[1] * inv))])
  #hand back the list of coordinates
  return decoded

In [18]:
# Load the path from Valhalla

import requests
import json

url = "http://valhalla:8002/route"
data =  {
    "locations":[
        {
            "lat":40.222884,
            "lon":-74.010232,
            "type":"break",
            "city":"Asbury Park",
            "state":"NJ"
        },{
            "lat":40.486678,
            "lon":-74.444414,
            "type":"break",
            "city":"New Brunswick",
            "state":"NJ"
        }
    ],
    "costing":"auto",
    "directions_options":{
        "units":"miles"
    }
}

data_json = json.dumps(data)
response = requests.post(url, data=data_json)
response_obj = json.loads(response.text)
distance = response_obj['trip']['summary']['length']
time_in_seconds = response_obj['trip']['summary']['time']
west = response_obj['trip']['summary']['min_lon']
south = response_obj['trip']['summary']['min_lat']
east = response_obj['trip']['summary']['max_lon']
north = response_obj['trip']['summary']['max_lat']
geometry = decode_geometry(response_obj['trip']['legs'][0]['shape'])
print (distance)
print (time_in_seconds)

34.854
2502


In [19]:
# Draw Path on a leaflet map

from ipyleaflet import Map, basemaps, basemap_to_tiles, Polyline

m = Map(
    layers=(basemap_to_tiles(basemaps.CartoDB.DarkMatter), ),
    center=((north+south)/2, (east+west)/2),
    zoom=10
)

line = Polyline(
    locations = geometry,
    color = "blue",
    fill = False)

m.add_layer(line)


m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …